# Radar LPDP Analytics Panel
# Interactive analysis tool untuk sample data radar menggunakan DearPyGUI

In [ ]:
# Import libraries
import sys
import os
from pathlib import Path

# Add parent directory to path untuk import functions
parent_dir = Path.cwd().parent
sys.path.insert(0, str(parent_dir))

import numpy as np
import dearpygui.dearpygui as dpg
from functions.data_processing import load_and_process_data, compute_fft, find_top_extrema
from config import SAMPLE_RATE, FFT_SMOOTHING_ENABLED, FFT_SMOOTHING_WINDOW

print("✅ Libraries loaded successfully")

In [ ]:
# Configuration
SAMPLE_DIR = Path("sample")
AVAILABLE_FILES = sorted(list(SAMPLE_DIR.glob("*.bin")))

print(f"📁 Found {len(AVAILABLE_FILES)} sample files:")
for f in AVAILABLE_FILES:
    print(f"  - {f.name}")

In [ ]:
# Global state untuk analytics panel
class AnalyticsState:
    def __init__(self):
        self.current_file_index = 0
        self.ch1_data = None
        self.ch2_data = None
        self.freqs_ch1 = None
        self.mag_ch1 = None
        self.freqs_ch2 = None
        self.mag_ch2 = None
        self.ch1_peaks = []
        self.ch2_peaks = []
        self.selected_peak_ch1 = None
        self.selected_peak_ch2 = None
        
state = AnalyticsState()

def load_current_file():
    """Load dan process file yang sedang dipilih"""
    if not AVAILABLE_FILES:
        return
    
    filepath = AVAILABLE_FILES[state.current_file_index]
    print(f"📂 Loading: {filepath.name}")
    
    # Load data
    ch1, ch2, n_samples, sr = load_and_process_data(str(filepath), SAMPLE_RATE)
    
    if ch1 is None:
        print(f"❌ Failed to load {filepath.name}")
        return
    
    state.ch1_data = ch1
    state.ch2_data = ch2
    
    # Compute FFT
    state.freqs_ch1, state.mag_ch1 = compute_fft(
        ch1, SAMPLE_RATE,
        smooth=FFT_SMOOTHING_ENABLED,
        smooth_window=FFT_SMOOTHING_WINDOW
    )
    state.freqs_ch2, state.mag_ch2 = compute_fft(
        ch2, SAMPLE_RATE,
        smooth=FFT_SMOOTHING_ENABLED,
        smooth_window=FFT_SMOOTHING_WINDOW
    )
    
    # Find peaks
    state.ch1_peaks, _ = find_top_extrema(state.freqs_ch1, state.mag_ch1, n_extrema=10)
    state.ch2_peaks, _ = find_top_extrema(state.freqs_ch2, state.mag_ch2, n_extrema=10)
    
    print(f"✅ Loaded {n_samples} samples, found {len(state.ch1_peaks)} CH1 peaks, {len(state.ch2_peaks)} CH2 peaks")
    
    return filepath.name

print("✅ State management ready")

In [ ]:
# Callback functions
def file_selector_callback(sender, app_data):
    """Callback ketika file dipilih dari combo"""
    selected_name = app_data
    # Find index
    for i, f in enumerate(AVAILABLE_FILES):
        if f.name == selected_name:
            state.current_file_index = i
            break
    
    # Load file
    load_current_file()
    update_all_plots()

def update_all_plots():
    """Update semua plot dengan data terbaru"""
    if state.ch1_data is None:
        return
    
    # Update FFT CH1
    if dpg.does_item_exist("fft_ch1_series"):
        dpg.set_value("fft_ch1_series", [state.freqs_ch1, state.mag_ch1])
        dpg.fit_axis_data("fft_ch1_xaxis")
        dpg.fit_axis_data("fft_ch1_yaxis")
    
    # Update FFT CH2
    if dpg.does_item_exist("fft_ch2_series"):
        dpg.set_value("fft_ch2_series", [state.freqs_ch2, state.mag_ch2])
        dpg.fit_axis_data("fft_ch2_xaxis")
        dpg.fit_axis_data("fft_ch2_yaxis")
    
    # Update peak tables
    update_peak_table("ch1", state.ch1_peaks)
    update_peak_table("ch2", state.ch2_peaks)

def update_peak_table(channel, peaks):
    """Update tabel peak untuk channel tertentu"""
    table_tag = f"{channel}_peak_table"
    
    if not dpg.does_item_exist(table_tag):
        return
    
    # Clear existing rows
    children = dpg.get_item_children(table_tag, slot=1)
    if children:
        for child in children:
            dpg.delete_item(child)
    
    # Add new rows
    for i, peak in enumerate(peaks[:10]):  # Max 10 peaks
        with dpg.table_row(parent=table_tag):
            dpg.add_text(f"{i+1}")
            dpg.add_text(f"{peak['index']}")
            dpg.add_text(f"{peak['freq_khz']:.2f}")
            dpg.add_text(f"{peak['mag_db']:.2f}")
            # Add button untuk hover info
            dpg.add_button(
                label="📍",
                width=30,
                callback=lambda s, a, u: highlight_peak(u),
                user_data=(channel, peak)
            )

def highlight_peak(user_data):
    """Highlight peak yang dipilih di plot"""
    channel, peak = user_data
    print(f"📍 {channel.upper()} Peak #{peak['index']}: {peak['freq_khz']:.2f} kHz, {peak['mag_db']:.2f} dB")

print("✅ Callbacks defined")

In [ ]:
# Create DearPyGUI Analytics Panel
def create_analytics_panel():
    """Buat panel analytics dengan DearPyGUI"""
    
    dpg.create_context()
    
    # Setup theme
    with dpg.theme() as global_theme:
        with dpg.theme_component(dpg.mvAll):
            dpg.add_theme_color(dpg.mvThemeCol_WindowBg, (15, 15, 15))
            dpg.add_theme_color(dpg.mvThemeCol_FrameBg, (30, 30, 30))
            dpg.add_theme_color(dpg.mvThemeCol_Button, (50, 50, 150))
            dpg.add_theme_color(dpg.mvThemeCol_ButtonHovered, (70, 70, 180))
            dpg.add_theme_style(dpg.mvStyleVar_FrameRounding, 5)
            dpg.add_theme_style(dpg.mvStyleVar_WindowPadding, 10, 10)
    
    dpg.bind_theme(global_theme)
    
    # Main window
    with dpg.window(label="Radar LPDP Analytics", tag="main_window", width=1400, height=900):
        
        # Top bar - File selector
        with dpg.group(horizontal=True):
            dpg.add_text("📁 Sample File:")
            dpg.add_combo(
                items=[f.name for f in AVAILABLE_FILES],
                default_value=AVAILABLE_FILES[0].name if AVAILABLE_FILES else "",
                callback=file_selector_callback,
                width=200,
                tag="file_selector"
            )
            dpg.add_button(label="🔄 Reload", callback=lambda: update_all_plots())
        
        dpg.add_separator()
        
        # Layout: 2 columns
        with dpg.group(horizontal=True):
            
            # Left column: FFT Plots
            with dpg.child_window(width=900, height=800):
                dpg.add_text("FFT Spectrum Analysis", color=(100, 200, 255))
                
                # CH1 Plot
                dpg.add_text("Channel 1 (CH1)", color=(255, 150, 100))
                with dpg.plot(label="CH1 FFT", height=350, width=-1):
                    dpg.add_plot_legend()
                    dpg.add_plot_axis(dpg.mvXAxis, label="Frequency (kHz)", tag="fft_ch1_xaxis")
                    dpg.add_plot_axis(dpg.mvYAxis, label="Magnitude (dB)", tag="fft_ch1_yaxis")
                    dpg.add_line_series(
                        [], [],
                        label="CH1",
                        parent="fft_ch1_yaxis",
                        tag="fft_ch1_series"
                    )
                
                dpg.add_spacer(height=10)
                
                # CH2 Plot
                dpg.add_text("Channel 2 (CH2)", color=(255, 200, 100))
                with dpg.plot(label="CH2 FFT", height=350, width=-1):
                    dpg.add_plot_legend()
                    dpg.add_plot_axis(dpg.mvXAxis, label="Frequency (kHz)", tag="fft_ch2_xaxis")
                    dpg.add_plot_axis(dpg.mvYAxis, label="Magnitude (dB)", tag="fft_ch2_yaxis")
                    dpg.add_line_series(
                        [], [],
                        label="CH2",
                        parent="fft_ch2_yaxis",
                        tag="fft_ch2_series"
                    )
            
            # Right column: Peak Tables
            with dpg.child_window(width=450, height=800):
                dpg.add_text("Peak Analysis", color=(100, 255, 150))
                
                # CH1 Peaks
                dpg.add_text("CH1 Top Peaks", color=(255, 150, 100))
                with dpg.table(
                    header_row=True,
                    borders_innerH=True,
                    borders_outerH=True,
                    borders_innerV=True,
                    borders_outerV=True,
                    tag="ch1_peak_table",
                    height=350
                ):
                    dpg.add_table_column(label="#", width_fixed=True, init_width_or_weight=30)
                    dpg.add_table_column(label="Index", width_fixed=True, init_width_or_weight=60)
                    dpg.add_table_column(label="Freq (kHz)", width_fixed=True, init_width_or_weight=100)
                    dpg.add_table_column(label="Mag (dB)", width_fixed=True, init_width_or_weight=80)
                    dpg.add_table_column(label="Info", width_fixed=True, init_width_or_weight=50)
                
                dpg.add_spacer(height=20)
                
                # CH2 Peaks
                dpg.add_text("CH2 Top Peaks", color=(255, 200, 100))
                with dpg.table(
                    header_row=True,
                    borders_innerH=True,
                    borders_outerH=True,
                    borders_innerV=True,
                    borders_outerV=True,
                    tag="ch2_peak_table",
                    height=350
                ):
                    dpg.add_table_column(label="#", width_fixed=True, init_width_or_weight=30)
                    dpg.add_table_column(label="Index", width_fixed=True, init_width_or_weight=60)
                    dpg.add_table_column(label="Freq (kHz)", width_fixed=True, init_width_or_weight=100)
                    dpg.add_table_column(label="Mag (dB)", width_fixed=True, init_width_or_weight=80)
                    dpg.add_table_column(label="Info", width_fixed=True, init_width_or_weight=50)
    
    # Setup viewport
    dpg.create_viewport(title="Radar LPDP Analytics", width=1420, height=920)
    dpg.setup_dearpygui()
    dpg.show_viewport()
    dpg.set_primary_window("main_window", True)
    
    # Load first file
    load_current_file()
    update_all_plots()
    
    print("✅ Analytics panel created!")
    print("🚀 Starting DearPyGUI render loop...")
    
    # Render loop
    while dpg.is_dearpygui_running():
        dpg.render_dearpygui_frame()
    
    dpg.destroy_context()
    print("👋 Analytics panel closed")

print("✅ Panel creator ready")

## 🚀 Run Analytics Panel

Jalankan cell di bawah untuk membuka interactive analytics panel.

**Features:**
- 📁 File selector untuk switch antar sample
- 📊 FFT spectrum visualization untuk CH1 & CH2
- 📈 Peak detection dengan tabel interaktif
- 🔍 Hover info untuk detail peak
- 🎨 Dark theme UI

**Controls:**
- Pilih file dari dropdown
- Klik 🔄 Reload untuk refresh data
- Klik 📍 di tabel untuk info detail peak

In [ ]:
# RUN THIS CELL to launch analytics panel
create_analytics_panel()

## 📊 Quick Analysis (Optional)

Cell di bawah untuk analisis cepat tanpa GUI (menggunakan print output):

In [ ]:
# Quick analysis tanpa GUI
def quick_analysis(filename):
    """Analisis cepat untuk satu file"""
    filepath = SAMPLE_DIR / filename
    
    if not filepath.exists():
        print(f"❌ File not found: {filename}")
        return
    
    print(f"\n{'='*60}")
    print(f"📊 QUICK ANALYSIS: {filename}")
    print(f"{'='*60}")
    
    # Load data
    ch1, ch2, n_samples, sr = load_and_process_data(str(filepath), SAMPLE_RATE)
    
    if ch1 is None:
        print("❌ Failed to load file")
        return
    
    print(f"📦 Samples: {n_samples}")
    print(f"⏱️  Duration: {n_samples/sr*1000:.2f} ms")
    print(f"🔊 Sample Rate: {sr/1e6:.1f} MHz")
    
    # Compute FFT
    freqs_ch1, mag_ch1 = compute_fft(ch1, SAMPLE_RATE, smooth=True)
    freqs_ch2, mag_ch2 = compute_fft(ch2, SAMPLE_RATE, smooth=True)
    
    # Find peaks
    ch1_peaks, _ = find_top_extrema(freqs_ch1, mag_ch1, n_extrema=5)
    ch2_peaks, _ = find_top_extrema(freqs_ch2, mag_ch2, n_extrema=5)
    
    print(f"\n📈 CH1 Top 5 Peaks:")
    print(f"{'#':<4} {'Index':<8} {'Freq (kHz)':<12} {'Mag (dB)':<10}")
    print("-" * 40)
    for i, p in enumerate(ch1_peaks, 1):
        print(f"{i:<4} {p['index']:<8} {p['freq_khz']:<12.2f} {p['mag_db']:<10.2f}")
    
    print(f"\n📈 CH2 Top 5 Peaks:")
    print(f"{'#':<4} {'Index':<8} {'Freq (kHz)':<12} {'Mag (dB)':<10}")
    print("-" * 40)
    for i, p in enumerate(ch2_peaks, 1):
        print(f"{i:<4} {p['index']:<8} {p['freq_khz']:<12.2f} {p['mag_db']:<10.2f}")
    
    print(f"\n{'='*60}\n")

# Example: Analyze all files
for f in AVAILABLE_FILES:
    quick_analysis(f.name)